In [1]:
import os

In [2]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig: 
    root_dir : Path 
    base_model_path : Path
    update_base_model_path : Path
    params_num_features : list 
    params_learning_rate : float
    params_include_top: bool
    params_weights: str
    params_classes : int

In [5]:
from fraud_prediction.constants import *
from fraud_prediction.utils.common import read_yaml, create_directories

In [6]:
import os 
from pathlib import Path

print(f"Current Path: {os.getcwd()}")

if os.getcwd().endswith("research"):
    os.chdir("../")

config_path = Path("config/config.yaml")
print(f"Config Path: {config_path.exists()}")

Current Path: c:\Users\artit\IKP_2025\fraud_prediction
Config Path: True


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            update_base_model_path=Path(config.update_base_model_path),
            params_num_features=self.params.NUM_FEATURES,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import xgboost as xg

[2026-01-22 18:14:33,548: INFO: utils: Note: NumExpr detected 22 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.]
[2026-01-22 18:14:33,548: INFO: utils: NumExpr defaulting to 16 threads.]


In [9]:
import tensorflow as tf
from pathlib import Path
import os
from fraud_prediction import logger

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        input_dim = self.config.params_num_features[0]
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(input_dim,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu')
        ])
        # เรียกใช้ save_model ผ่าน self
        self.save_model(path=self.config.base_model_path, model=self.model)

    def update_base_model(self):
        full_model = tf.keras.models.Sequential([
            self.model,
            tf.keras.layers.Dense(self.config.params_classes, activation='sigmoid')
        ])
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
        full_model.summary()
        self.save_model(path=self.config.update_base_model_path, model=full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        path.parent.mkdir(parents=True, exist_ok=True)
        model.save(str(path))
        logger.info(f"Model saved at: {path}")

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    print("Model Innitialization and Update Completed")
except Exception as e:
    raise e

[2026-01-22 18:14:36,196: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-22 18:14:36,197: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-22 18:14:36,197: INFO: common: created directory at: artifacts]
[2026-01-22 18:14:36,201: INFO: common: created directory at: artifacts/prepare_base_model]
[2026-01-22 18:14:36,246: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2026-01-22 18:14:36,277: INFO: 1034978456: Model saved at: artifacts\prepare_base_model\base_model.h5]


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 32)             │         2,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,625 (10.25 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

[2026-01-22 18:14:36,305: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2026-01-22 18:14:36,317: INFO: 1034978456: Model saved at: artifacts\prepare_base_model\base_model_updated.h5]
Model Innitialization and Update Completed
